# Ejemplo de visualización usando Naive-Bayes

## Preliminar

In [1]:
import sys
sys.path.insert(0, '..')
from utils.preprocesamiento import StemmerTokenizer, make_BoW_preprocess


tokenizador = StemmerTokenizer(stem=False)

# preprocessing = make_BoW_preprocess(tokenizador,'comment',max_ngram=1,min_ngram=1,mindf=10)

from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_selection import SelectPercentile, f_classif
from sklearn.pipeline import Pipeline

"""bog_pipeline = Pipeline(
    [("Preprocessing", preprocessing),
    # ("Selection", SelectPercentile(f_classif, percentile=90)),
    ("Clasificador", MultinomialNB())]
)""";

from utils.cargar import df_caso
from utils.preprocesamiento import process_df, procesar_adela

caso = 'adela'
df = df_caso(caso)

df = procesar_adela(df)
df = df[df['opt_left'] == 'Producir el alimento contra déficit vitamínico']

df = process_df(df,'comment','sel',verbose=False)

df = df.drop(columns=['user_id','team_id','gender','df','title','opt_left','opt_right','max_num','phase','time','curso'])

from sklearn.model_selection import train_test_split

# df_train, df_test, y_train, y_test = train_test_split(df, df['sel'], test_size=.25, stratify=df['sel'], random_state=0)
# bog_pipeline.fit(df_train,y_train)

df_bin = df.copy()

df_bin.loc[(df_bin['sel']==2) | (df_bin['sel']==3),'sel'] = 1
df_bin.loc[(df_bin['sel']==4) | (df_bin['sel']==5),'sel'] = 6

df_train_binary, df_test_binary, y_train_binary, y_test_binary = train_test_split(df_bin, df_bin['sel'], test_size=.25, stratify=df_bin['sel'], random_state=0)

preprocessing = make_BoW_preprocess(tokenizador,'comment',max_ngram=2,min_ngram=1,mindf=10)

bog_pipeline_binary = Pipeline(
    [("Preprocessing", preprocessing),
    # ("Selection", SelectPercentile(f_classif, percentile=90)),
    ("Clasificador", MultinomialNB())]
)

bog_pipeline_binary.fit(df_train_binary,y_train_binary)

/home/camilo/miniconda3/envs/ethics_env/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Pipeline(steps=[('Preprocessing',
                 ColumnTransformer(transformers=[('bag-of-words',
                                                  CountVectorizer(min_df=10,
                                                                  ngram_range=(1,
                                                                               2),
                                                                  tokenizer=<utils.preprocesamiento.StemmerTokenizer object at 0x7f7ca41998b0>),
                                                  'comment')])),
                ('Clasificador', MultinomialNB())])

In [2]:
from NB_utils import get_probs

vectorizer = bog_pipeline_binary.named_steps['Preprocessing'].transformers_[0][1]

prob_1 = {}

for ngram in vectorizer.get_feature_names_out():
    probs = get_probs(ngram,bog_pipeline_binary,df_test_binary,'comment')
    prob_1[ngram] = probs[0]

# op_left, op_right = 'Producir el alimento', 'Resguardar tradiciones/recurso'

In [3]:
def probability(token):
    try:
        return prob_1[token]
    except KeyError:
        return None

In [4]:
from utils.visualisation import HighlightedText

example_text = "creo que las creencias religiosas o tradiciones no deberian interferir en los avances cientificos"
ht = HighlightedText(example_text,range_values=(0,1))

ht.set_values(probability)

In [5]:
from IPython.display import display, HTML

display(HTML(ht.render()))